In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


En este notebook implementamos un asistente virtual para interactuar con mi blog: https://www.sensiocoders.com/blog

In [92]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm
import pinecone
from dotenv import load_dotenv

load_dotenv()

True

## Scrapping

Recuperar lista de posts

In [91]:
url = "https://www.sensiocoders.com/blog/"  
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
posts = soup.find_all('a', href=lambda href: href and "/blog/" in href)
len(posts)

112

 Para cada post, extraemos el contenido

In [93]:
for post in tqdm(posts):
	post_url = post['href'].split('/')[-1]
	post_response = requests.get(url + post_url)
	post_soup = BeautifulSoup(post_response.content, 'html.parser')
	content = post_soup.find('div', class_="post").text 
	os.makedirs('posts', exist_ok=True)
	with open(f'posts/{post_url}.txt', 'w') as f:
		f.write(content)

100%|██████████| 112/112 [00:27<00:00,  4.01it/s]


## Embedding

Generamos los embeddings de cada post y los guardamos en vector db.

In [55]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma, Pinecone

In [101]:
def read_post(post):
    with open('posts/' + post, 'r') as f:
        return f.read()
    
posts = [read_post(post) for post in os.listdir('posts')]

len(posts)

112

In [102]:
import numpy as np

lens = [len(post) for post in posts]

np.mean(lens), np.std(lens), np.min(lens), np.max(lens)

(15895.99107142857, 7516.431447520654, 5016, 37775)

Cortamos los posts en trozos de tamaño fijado para poder pasarselo al modelo sin pasarnos del límite de caracteres.

In [103]:
from langchain.text_splitter import CharacterTextSplitter

# text_splitter = CharacterTextSplitter(chunk_size=8192, chunk_overlap=256)
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=3000, chunk_overlap=0, disallowed_special=()
)

metadatas = [{"source": post} for post in os.listdir('posts')]
documents = text_splitter.create_documents(posts, metadatas=metadatas)

len(documents)


Created a chunk of size 3149, which is longer than the specified 3000
Created a chunk of size 6625, which is longer than the specified 3000
Created a chunk of size 3229, which is longer than the specified 3000


Created a chunk of size 3257, which is longer than the specified 3000
Created a chunk of size 7452, which is longer than the specified 3000
Created a chunk of size 3849, which is longer than the specified 3000
Created a chunk of size 5248, which is longer than the specified 3000


316

Los embeddings de `HuggingFaceEmbeddings` son gratis, de tamaño 768, pero no me han funcionado del todo bien. Los de `OpenAIEmbeddings` son de pago, de tamaño 1536, pero funcionan mejor.

In [6]:
# embeddings = HuggingFaceEmbeddings() 
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv('OPENAI_API_KEY'), disallowed_special=())

text = "Hola que tal?"

query_result = embeddings.embed_query(text)

len(query_result)

1536

Podemos usar `Chroma` para guardar los embeddings en local.

In [111]:
# comentado para evitar costes
# vectorstore = Chroma.from_documents(documents, embeddings)

> Docs: https://api.python.langchain.com/en/latest/modules/vectorstores.html

Alternativamente, `Pinecone` nos permite guardar los embeddings en la nube (gratis, con límites).

In [109]:
pinecone.init(api_key=os.environ['PINECONE_API_KEY'], environment=os.environ['PINECONE_ENVIRONMENT'])

# index = pinecone.Index('blog-qa') 
# delete_response = index.delete(deleteAll=True)

In [110]:
# generar la primera vez
# vectorstore = Pinecone.from_documents(documents, embeddings, index_name='blog-qa')

# una vez generados, se pueden cargar
vectorstore = Pinecone.from_existing_index('blog-qa', embeddings)

In [112]:
query = 'que es el PBDL?'

docs = vectorstore.similarity_search(query, k=3)

docs

[Document(page_content='Physics-Based Deep Learning\nIntroducción\nEste es el primero en una serie de posts en los que aprenderemos sobre PBDL: Physics-Based Deep Learning, o el uso del Deep Learning (redes neuronales) para simulación física. En concreto, nos centraremos en CFD: Computational Fluid Dynamics, el campo de la física que se enfoca en la simulación de fluidos para aplicaciones de aerodinámica, combustión, etc. Te advierto que nos vamos a alejar del machine learning tradicional para explorar un nuevo campo, el del uso de las redes neuronales para aproximar soluciones a ecuaciones diferenciales. Es posible que en algunos momentos te preguntes: ¿es esto realmente machine learning? Te entiendo. Aún así, creo firmemente que el campo del PBDL revolucionará la manera en la que simulamos la naturaleza en los próximos años, de la misma manera que el Deep Learning ha revolucionado (y lo sigue haciendo) tantos otros campos de la ciencia, como por ejemplo el plegado de proteinas.\nEl c

## QA pipeline

### The LLM model

Una vez preparados los datos, procedemos a instanciar el LLM que queramos usar.

In [137]:
from langchain import HuggingFacePipeline, HuggingFaceHub, OpenAI
import torch

model = "tiiuae/falcon-40b-instruct"

# modelos HF en local
# llm = HuggingFacePipeline.from_model_id(
#     model_id=model, 
#     task="text-generation", 
#     model_kwargs={
#         "max_length": 2048, # debe incluir documentos
#         'device_map': 'auto',
#         'trust_remote_code': True,
#         'torch_dtype': torch.bfloat16
# 	}
# )

# modelos HF en la nube
# llm = HuggingFaceHub(repo_id=model, model_kwargs={"temperature": 0.1, "max_length": 2048})

# modelos OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0, openai_api_key=os.environ['OPENAI_API_KEY'], max_tokens=256)


/home/juan/.local/lib/python3.10/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/juan/.local/lib/python3.10/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Prompts

Opcionalmente, podemos modificar los prompts para que se adapten mejor a nuestro caso de uso.

In [138]:
from langchain.prompts.prompt import PromptTemplate

# este es el prompt que se usará para generar las respuestas

template = """Dado el siguiente contexto, reponde la pregunta.
Si no sabes la respuesta, responde con "No lo sé". 
No inventes respuestas.
Responde siempre en español.

Contexto:

{context} 

Pregunta:

{question}"""

prompt = PromptTemplate(template=template, input_variables=['context', 'question'])

In [139]:
# este es el prompt que se usará cuando usemos contexto de chat (el prompt anterior se ejectuará después)

template = """Dado el siguiente historial de conversación, reformula la pregunta de forma que el modelo pueda responderla:

Historial:

{chat_history}

Pregunta:

{question}

Progunta reformulada:"""

condense_question_prompt = PromptTemplate.from_template(template)

Creamos la `chain`

In [140]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm, 
    vectorstore.as_retriever(search_kwargs={'k': 1}), 
    return_source_documents=True, 
    condense_question_prompt=condense_question_prompt, 
    combine_docs_chain_kwargs={'prompt': prompt}
)

### Chat

In [141]:
chat_history = []
query = "Que es el PBDL?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'El PBDL es el uso del Deep Learning (redes neuronales) para simulación física, en concreto, se centra en CFD: Computational Fluid Dynamics, el campo de la física que se enfoca en la simulación de fluidos para aplicaciones de aerodinámica, combustión, etc.'

In [142]:
[source.metadata['source'] for source in result['source_documents']]

['080_pbdl_intro.txt']

In [143]:
chat_history.append((query, result["answer"]))
query = "Que ventajas ofrece?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'El uso del PBDL en la simulación de fluidos para aplicaciones de aerodinámica, combustión, etc. supone una revolución y acelerará, a la vez que abaratará, todo el proceso de diseño de vehículos, dando como resultado vehículos más eficientes, que viajen más rápido consumiendo y contaminando menos.'

In [144]:
[source.metadata['source'] for source in result['source_documents']]

['080_pbdl_intro.txt']

Ya lo tenemos todo listo para implementar una API con la que interactuar con el blog a través d elenguaje natural (ver script adjunto)

## Ideas de mejoras

- Agregar transcripciones de vídeos de youtube
- Agregar funcionalidad de voz
- Automatizar ETL (cronjob)